In [ ]:
# source /opt/envvars.sh

!pip3 install python-dotenv
%load_ext dotenv
%dotenv -o /opt/envvars.sh
%env

# YARN

## YARN - Web interface

- Master node
    - Resource Manager: http://localhost:8088
    - Timeline Service: http://localhost:8188
- Worker node
    - hadoop1
        - NodeManager: http://localhost:8042
    - hadoop2
        - NodeManager: http://localhost:8043
    - hadoop3
        - NodeManager: http://localhost:8044

## Hadoop MapReduce Examples

```
$HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar
```

- aggregatewordcount: An Aggregate based map/reduce program that counts the words in the input files.
- aggregatewordhist: An Aggregate based map/reduce program that computes the histogram of the words in the input files.
- bbp: A map/reduce program that uses Bailey-Borwein-Plouffe to compute exact digits of Pi.
- dbcount: An example job that count the pageview counts from a database.
- distbbp: A map/reduce program that uses a BBP-type formula to compute exact bits of Pi.
- grep: A map/reduce program that counts the matches of a regex in the input.
- join: A job that effects a join over sorted, equally partitioned datasets
- multifilewc: A job that counts words from several files.
- pentomino: A map/reduce tile laying program to find solutions to pentomino problems.
- pi: A map/reduce program that estimates Pi using a quasi-Monte Carlo method.
- randomtextwriter: A map/reduce program that writes 10GB of random textual data per node.
- randomwriter: A map/reduce program that writes 10GB of random data per node.
- secondarysort: An example defining a secondary sort to the reduce.
- sort: A map/reduce program that sorts the data written by the random writer.
- sudoku: A sudoku solver.
- teragen: Generate data for the terasort
- terasort: Run the terasort
- teravalidate: Checking results of terasort
- wordcount: A map/reduce program that counts the words in the input files.
- wordmean: A map/reduce program that counts the average length of the words in the input files.
- wordmedian: A map/reduce program that counts the median length of the words in the input files.
- wordstandarddeviation: A map/reduce program that counts the standard deviation of the length of the words in the input files.


In [ ]:
%%bash

cd /opt/hadoop/share/hadoop/mapreduce

# hadoop jar ./hadoop-mapreduce-examples-3.2.3.jar

# randomwriter: A map/reduce program that writes 10GB of random data per node
# configured for 150MB of random text / 50 MB per map (3 maps)
hadoop jar ./hadoop-mapreduce-examples-3.2.3.jar randomtextwriter \
  -D mapreduce.randomtextwriter.totalbytes=157286400 \
  -D mapreduce.randomtextwriter.bytespermap=52428800 \
  -D mapreduce.output.fileoutputformat.compress=false \
  -outFormat org.apache.hadoop.mapreduce.lib.output.TextOutputFormat \
  randomtext

# wordcount: A map/reduce program that counts the words in the input files
hadoop jar ./hadoop-mapreduce-examples-3.2.3.jar wordcount randomtext randomtextcount



In [ ]:
%%bash

cd /opt/hadoop/share/hadoop/mapreduce

# Teragen - 200MB
hadoop jar ./hadoop-mapreduce-examples-3.2.3.jar teragen \
  2000000 \
  teragenoutput

# Terasort
hadoop jar ./hadoop-mapreduce-examples-3.2.3.jar terasort \
  teragenoutput terasortoutput

## YARN - CLI

- https://hadoop.apache.org/docs/r3.2.3/hadoop-yarn/hadoop-yarn-site/YarnCommands.html

In [ ]:
%%bash

yarn help


### List cluster node status

In [ ]:
%%bash

yarn node -all -list -showDetails



### List applications

In [ ]:
%%bash

# Current applications (running or accepted)
# yarn app -list
# Applications already executed
yarn app -list -appStates ALL


### List queue status

In [ ]:
%%bash

yarn queue -status default

### Get application log

In [ ]:
%%bash

APPID=$(yarn application -list -appStates FINISHED 2>/dev/null | tail -1 | awk '{ print $1 }')
echo "Getting logs for $APPID"
yarn logs -applicationId $APPID 2>/dev/null | head -n 50

### Kill application

In [ ]:
%%bash

APPID=$(yarn application -list -appStates RUNNING 2>/dev/null | tail -1 | awk '{ print $1 }')
echo "Killing $APPID"
yarn application -kill $APPID

### Handling nodemanager failures

- yarn.nm.liveness-monitor.expiry-interval-ms property in yarn-site.xml
 - set to 10000 (10 seconds) / default is 600000 (10 minutes)

In [ ]:
%%bash

# simulate node fault
ssh hadoop1 'kill -9 $(cat /tmp/hadoop-hadoop-nodemanager.pid)'

- http://localhost:8088/cluster/nodes
- Wait 10 seconds to discover node failure

In [ ]:
%%bash

yarn node -list -showDetails

In [ ]:
%%bash

# Restart nodemanager
ssh hadoop1 /opt/hadoop/bin/yarn --daemon start nodemanager